#### Tests before extension install

In [1]:
integer = 23
integer

23

In [2]:
integer = "works fine"
integer

'works fine'

## Install and load extension

In [3]:
!pip install -I git+https://github.com/alanbraz/ipython-ariadne-extension.git

  Cloning https://github.com/alanbraz/ipython-ariadne-extension.git to /gpfs/global_fs01/sym_shared/YPProdSpark/user/saa6-1973990e46b9ea-cc8e51e960b7/notebook/tmp/pip-_uyhdduo-build
  Running setup.py install for ipython-ariadne-extension ... done


In [4]:
%load_ext ariadne

#### Tests OK

In [5]:
1+1

2

Adriane DEBUG: check OK!


In [6]:
1+"123"

TypeError: unsupported operand type(s) for +: 'int' and 'str'

Adriane DEBUG: check OK!


In [7]:
from __future__ import division, print_function, absolute_import

# Import MNIST data
from tensorflow.examples.tutorials.mnist import input_data
mnist = []
try:
    mnist = input_data.read_data_sets("/tmp/data/", one_hot=False)
except:
    exit

import tensorflow as tf

# Training Parameters
learning_rate = 0.001
num_steps = 2000
batch_size = 128

# Network Parameters
num_input = 784 # MNIST data input (img shape: 28*28)
num_classes = 10 # MNIST total classes (0-9 digits)
dropout = 0.75 # Dropout, probability to keep units

# Create the neural network
def conv_net(x_dict, n_classes, dropout, reuse, is_training):
    # Define a scope for reusing the variables
    with tf.variable_scope('ConvNet', reuse=reuse):
        # TF Estimator input is a dict, in case of multiple inputs
        x = x_dict['images']

        bad_x = tf.reshape(x, shape=[-1, 11, 28, 1])

        # MNIST data input is a 1-D vector of 784 features (28*28 pixels)
        # Reshape to match picture format [Height x Width x Channel]
        # Tensor input become 4-D: [Batch Size, Height, Width, Channel]
        z = tf.reshape(x, shape=[-1, 28, 28, 1])

        # Convolution Layer with 32 filters and a kernel size of 5
        conv1 = tf.layers.conv2d(z, 32, 5, activation=tf.nn.relu)
        # Max Pooling (down-sampling) with strides of 2 and kernel size of 2
        conv1 = tf.layers.max_pooling2d(conv1, 2, 2)

        # Convolution Layer with 64 filters and a kernel size of 3
        conv2 = tf.layers.conv2d(conv1, 64, 3, activation=tf.nn.relu)
        # Max Pooling (down-sampling) with strides of 2 and kernel size of 2
        conv2 = tf.layers.max_pooling2d(conv2, 2, 2)

        bad_conv1 = tf.layers.conv2d(x, 32, 5, activation=tf.nn.relu)

        # Flatten the data to a 1-D vector for the fully connected layer
        fc1 = tf.contrib.layers.flatten(conv2)

        # Fully connected layer (in tf contrib folder for now)
        fc1 = tf.layers.dense(fc1, 1024)
        # Apply Dropout (if is_training is False, dropout is not applied)
        fc1 = tf.layers.dropout(fc1, rate=dropout, training=is_training)

        # Output layer, class prediction
        out = tf.layers.dense(fc1, n_classes)

    return out

# Build the neural network
# Define the model function (following TF Estimator Template)
def model_fn(features, labels, mode):
    # Because Dropout have different behavior at training and prediction time, we
    # need to create 2 distinct computation graphs that still share the same weights.
    logits_train = conv_net(features, num_classes, dropout, reuse=False,
                            is_training=True)
    logits_test = make_net(features, num_classes, dropout, reuse=True,
                           is_training=False)

    # Predictions
    pred_classes = tf.argmax(logits_test, axis=1)
    pred_probas = tf.nn.softmax(logits_test)

    # If prediction mode, early return
    if mode == tf.estimator.ModeKeys.PREDICT:
        return tf.estimator.EstimatorSpec(mode, predictions=pred_classes)

        # Define loss and optimizer
    loss_op = tf.reduce_mean(tf.nn.sparse_softmax_cross_entropy_with_logits(
        logits=logits_train, labels=tf.cast(labels, dtype=tf.int32)))
    optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate)
    train_op = optimizer.minimize(loss_op,
                                  global_step=tf.train.get_global_step())

    # Evaluate the accuracy of the model
    acc_op = tf.metrics.accuracy(labels=labels, predictions=pred_classes)

    # TF Estimators requires to return a EstimatorSpec, that specify
    # the different ops for training, evaluating, ...
    estim_specs = tf.estimator.EstimatorSpec(
        mode=mode,
        predictions=pred_classes,
        loss=loss_op,
        train_op=train_op,
        eval_metric_ops={'accuracy': acc_op})

    return estim_specs

make_net = conv_net

# Build the Estimator
model = tf.estimator.Estimator(model_fn)

# Define the input function for training
input_fn = tf.estimator.inputs.numpy_input_fn(
    x={'images': mnist.train.images}, y=mnist.train.labels,
    batch_size=batch_size, num_epochs=None, shuffle=True)
# Train the Model
model.train(input_fn, steps=num_steps)

# Evaluate the Model
# Define the input function for evaluating
input_fn = tf.estimator.inputs.numpy_input_fn(
    x={'images': mnist.test.images}, y=mnist.test.labels,
    batch_size=batch_size, shuffle=False)
# Use the Estimator 'evaluate' method
e = model.evaluate(input_fn)

print("Testing Accuracy:", e['accuracy'])


INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_save_checkpoints_steps': None, '_log_step_count_steps': 100, '_save_checkpoints_secs': 600, '_tf_random_seed': 1, '_session_config': None, '_save_summary_steps': 100, '_keep_checkpoint_every_n_hours': 10000, '_model_dir': '/gpfs/fs01/user/saa6-1973990e46b9ea-cc8e51e960b7/notebook/tmp/tmpyfwgxdq5', '_keep_checkpoint_max': 5}


AttributeError: 'list' object has no attribute 'train'

Adriane diagnostic error: Bad type to convolve pixel[n][28 * 28], needs 4 dimensions (possible fix: tf.reshape(x, [-1, 28, 28, 1]))
Line 46:         bad_conv1 = tf.layers.conv2d(x, 32, 5, activation=tf.nn.relu)
                                              ^


In [8]:
base_init = 10

def id(x):
    return x

def call(x, y):
    return x(y)

def foo(a,b):
    return call(id, a+b)

class Foo(object):
    base = base_init
    
    def foo(self, a, b):
        self.contents = id(a+b+self.base)
        return self.contents

print(Foo)

print(Foo.foo)
print(Foo.base)

print(foo)
print(foo(1,2))

instance = Foo()
print(Foo.foo(instance, 2,3))
print(instance.foo(2,3))

f = instance.foo
print(f);
print(f(3,4))

instance.f = foo
print(instance.f(4,5))
print(instance.f);

instance.foo = foo;
print(instance.foo(5,6))
print(instance.foo);

foo.x = foo;
print(foo.x(6,7));
print(foo.x);

x = Foo
print(x)
y = x()
print(y)
print(y.foo(7,8))

def nothing():
    return 0

z = id(nothing)
z()


<class '__main__.Foo'>
<function Foo.foo at 0x7fb8a6b907b8>
10
<function foo at 0x7fb8a6b906a8>
3
15
15
<bound method Foo.foo of <__main__.Foo object at 0x7fb8a6b9b9e8>>
17
9
<function foo at 0x7fb8a6b906a8>
11
<function foo at 0x7fb8a6b906a8>
13
<function foo at 0x7fb8a6b906a8>
<class '__main__.Foo'>
25


0

Adriane DEBUG: check OK but with warnings!


In [9]:
def f(x: int) -> int:
     return x
print("type annotation of function f: " + str(f.__annotations__))
print("result of calling f('ab'): " + str(f('ab')))

# alanbraz: something is breaking the HTTP body post, maybe the arrown?!!?

type annotation of function f: {'return': <class 'int'>, 'x': <class 'int'>}
result of calling f('ab'): ab


Adriane ERROR calling IBM Function: <class 'urllib.error.HTTPError'>
Traceback (most recent call last):
  File "/gpfs/fs01/user/saa6-1973990e46b9ea-cc8e51e960b7/.local/lib/python3.5/site-packages/ariadne/ariadne.py", line 44, in check
    reply = urllib.request.urlopen(req, jsondataasbytes).read()
  File "/usr/local/src/conda3_runtime/home/envs/DSX-Python35-Spark/lib/python3.5/urllib/request.py", line 163, in urlopen
    return opener.open(url, data, timeout)
  File "/usr/local/src/conda3_runtime/home/envs/DSX-Python35-Spark/lib/python3.5/urllib/request.py", line 472, in open
    response = meth(req, response)
  File "/usr/local/src/conda3_runtime/home/envs/DSX-Python35-Spark/lib/python3.5/urllib/request.py", line 582, in http_response
    'http', request, response, code, msg, hdrs)
  File "/usr/local/src/conda3_runtime/home/envs/DSX-Python35-Spark/lib/python3.5/urllib/request.py", line 510, in error
    return self._call_chain(*args)
  File "/usr/local/src/conda3_runtime/home/envs/DSX